# Clustering Neighbourhood in Toronto
### Part2

In [1]:
from urllib.request import urlopen as uReq
from  bs4 import BeautifulSoup as soup
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn import metrics
import folium 
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

print('Libraries imported.')

Libraries imported.


## **Web Scraping Wikipedia**

In [2]:
pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969')

dfs = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969', header=0)

In [3]:
df = dfs[0]
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [4]:
# Cleaning data by filtering all the Not assigned data values
df_filtered = df[df['Borough'] != 'Not assigned']
df_filtered.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
# Now we would have a look at dataframe shape by using the .shape function
df_filtered.shape

(103, 3)

### Importing the Geospatial Coordinates data

In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_156f8cbe0974441e9631771cd64b46ee = 'https://s3.eu-geo.objectstorage.softlayer.net'
else:
    endpoint_156f8cbe0974441e9631771cd64b46ee = 'https://s3.eu-geo.objectstorage.service.networklayer.com'

client_156f8cbe0974441e9631771cd64b46ee = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='BDka88JqBcYq5XcOZrwCpmdwD3VsgSt_gAsUHwv7Mqpl',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_156f8cbe0974441e9631771cd64b46ee)

body = client_156f8cbe0974441e9631771cd64b46ee.get_object(Bucket='applieddatasciencecapstone-donotdelete-pr-n6sddztxqqvlxr',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df_data_1.shape
#As shown by the .shape function, the Geospatial_Coordinates.csv has 103 rows and 3 columns too! It matches the filtered dataframe .shape function.

(103, 3)

### Now we will merge the two dataframes.


In [8]:
neighbourhoods = df_filtered.merge(df_data_1,left_on='Postal Code',right_on='Postal Code')
neighbourhoods.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
